# 新浪股市汇热度

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [2]:
r = requests.get("http://guba.sina.com.cn/?s=bar&bid=14247")

In [3]:
r.status_code

200

In [4]:
r.encoding

'gbk'

In [5]:
soup = BeautifulSoup(r.text, "html.parser")
table_1, table_2 = soup.find_all("div",{"class":"b_cont"})[1].find_all("table")

In [6]:
# 个股热度
print(table_1.prettify())

<table cellpadding="0" cellspacing="0" class="table_01" id="hotrank_cont_01" width="100%">
 <tr>
  <th class="fisrt_th" width="25">
   公司
  </th>
  <th width="90">
  </th>
  <th class="ta_left" width="70">
   热度
  </th>
  <th>
   趋势
  </th>
 </tr>
 <tr>
  <td>
   <em class="number_em_01">
    1
   </em>
  </td>
  <td class="ta_left">
   <a href="/?s=bar&amp;name=sz002543" target="_blank">
    万和电气
   </a>
  </td>
  <td class="color_red ta_left">
   54
  </td>
  <td>
   <em class="icon_em em_arrow_up">
    up
   </em>
  </td>
 </tr>
 <tr>
  <td>
   <em class="number_em_01">
    2
   </em>
  </td>
  <td class="ta_left">
   <a href="/?s=bar&amp;name=sh600301" target="_blank">
    ST南化
   </a>
  </td>
  <td class="color_red ta_left">
   39
  </td>
  <td>
   <em class="icon_em em_arrow_up">
    up
   </em>
  </td>
 </tr>
 <tr>
  <td>
   <em class="number_em_01">
    3
   </em>
  </td>
  <td class="ta_left">
   <a href="/?s=bar&amp;name=sz000732" target="_blank">
    泰禾集团
   </a>
  </td>
  <

In [7]:
# 主题热度
print(table_2.prettify())

<table cellpadding="0" cellspacing="0" class="table_01" id="hotrank_cont_02" width="100%">
 <tr>
  <th class="fisrt_th" width="25">
   公司
  </th>
  <th width="90">
  </th>
  <th class="ta_left" width="70">
   热度
  </th>
  <th>
   趋势
  </th>
 </tr>
 <tr>
  <td>
   <em class="number_em_01">
    1
   </em>
  </td>
  <td class="ta_left">
   <a href="/?s=bar&amp;name=qh" target="_blank">
    期货
   </a>
  </td>
  <td class="color_red ta_left">
   189
  </td>
  <td>
   <em class="icon_em em_arrow_up">
    up
   </em>
  </td>
 </tr>
 <tr>
  <td>
   <em class="number_em_01">
    2
   </em>
  </td>
  <td class="ta_left">
   <a href="/?s=bar&amp;name=%B9%C9%CA%D0%CA%B5%D5%BD" target="_blank">
    股市实战
   </a>
  </td>
  <td class="color_red ta_left">
   52.5
  </td>
  <td>
   <em class="icon_em em_arrow_up">
    up
   </em>
  </td>
 </tr>
 <tr>
  <td>
   <em class="number_em_01">
    3
   </em>
  </td>
  <td class="ta_left">
   <a href="/?s=bar&amp;name=%C3%FB%BC%D2%C2%DB%CA%D0" target="_blank">
 

In [8]:
def parse_table(table, label):
    """ label: stock 个股
                theme 主题 """
    for tr in table.find_all("tr"):
        try:
            a = tr.find("td", {"class": 'ta_left'}).a
            name = a.text
            hot = tr.find("td", {"class": "color_red ta_left"}).text
            if label == "stock":
                code = re.findall(r"s[hz]\d{6}", a.attrs["href"])[0]
                yield code, name, hot
            if label == "theme":
                 yield name, hot       
        except:
            continue   

In [9]:
codes, names, hots = zip(*list(parse_table(table_1, "stock")))
df_1 = pd.DataFrame({'代码': codes, "名称": names, "热度": hots})
df_1['热度'] = df_1['热度'].astype(float)
df_1.index += 1
df_1

,代码,名称,热度
1,sz002543,万和电气,54.0
2,sh600301,ST南化,39.0
3,sz000732,泰禾集团,34.5
4,sz000980,众泰汽车,33.0
5,sz002477,*ST雏鹰,33.0
6,sz300786,国林环保,30.0
7,sz000725,京东方A,28.5
8,sz002201,九鼎新材,24.0
9,sz000413,东旭光电,22.5
10,sz002450,*ST康得,22.5


In [10]:
names, hots = zip(*list(parse_table(table_2, "theme")))
df_2 = pd.DataFrame({"名称": names, "热度": hots})
df_2['热度'] = df_2['热度'].astype(float)
df_2.index += 1
df_2

,名称,热度
1,期货,189.0
2,股市实战,52.5
3,名家论市,37.5
4,环球老虎,12.0
5,财经建议,9.0
6,股民学校,8.0
7,全球指数,6.0
8,涨停早知,5.0
9,技术分析,5.0
10,彬哥看盘,5.0
